In [ ]:
#Defining the model

model = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
output_feat=args.output_size,seq_len=args.obs_seq_len,
kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)


#Training settings

optimizer = optim.SGD(model.parameters(),lr=args.lr)

if args.use_lrschd:
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_sh_rate, gamma=0.1)



checkpoint_dir = './checkpoint/'+args.tag+'/'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

with open(checkpoint_dir+'args.pkl', 'wb') as fp:
    pickle.dump(args, fp)



print('Data and model loaded')
print('Checkpoint dir:', checkpoint_dir)

#Training
metrics = {'train_loss':[],  'val_loss':[]}
constant_metrics = {'min_val_epoch':-1, 'min_val_loss':9999999999999999}

def train(epoch):
    global metrics,loader_train
    model.train()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_train)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1


    for cnt,batch in enumerate(loader_train):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        optimizer.zero_grad()
        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)



        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            loss.backward()

            if args.clip_grad is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip_grad)


            optimizer.step()
            #Metrics
            loss_batch += loss.item()
            print('TRAIN:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['train_loss'].append(loss_batch/batch_count)




def vald(epoch):
    global metrics,loader_val,constant_metrics
    model.eval()
    loss_batch = 0
    batch_count = 0
    is_fst_loss = True
    loader_len = len(loader_val)
    turn_point =int(loader_len/args.batch_size)*args.batch_size+ loader_len%args.batch_size -1

    for cnt,batch in enumerate(loader_val):
        batch_count+=1

        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs, V_tr,A_tr,A_dir_tr = batch


        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(), A_dir_obs.squeeze())

        V_pred = V_pred.permute(0,2,3,1)

        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()

        if batch_count%args.batch_size !=0 and cnt != turn_point :
            l = graph_loss(V_pred,V_tr)
            if is_fst_loss :
                loss = l
                is_fst_loss = False
            else:
                loss += l

        else:
            loss = loss/args.batch_size
            is_fst_loss = True
            #Metrics
            loss_batch += loss.item()
            print('VALD:','\t Epoch:', epoch,'\t Loss:',loss_batch/batch_count)

    metrics['val_loss'].append(loss_batch/batch_count)

    if  metrics['val_loss'][-1]< constant_metrics['min_val_loss']:
        constant_metrics['min_val_loss'] =  metrics['val_loss'][-1]
        constant_metrics['min_val_epoch'] = epoch
        torch.save(model.state_dict(),checkpoint_dir+'val_best.pth')  # OK
        check_test_performance()

print('Training started ...')
for epoch in range(args.num_epochs):
    train(epoch)
    vald(epoch)
    if args.use_lrschd:
        scheduler.step()


    print('*'*30)
    print('Epoch:',args.tag,":", epoch)
    for k,v in metrics.items():
        if len(v)>0:
            print(k,v[-1])


    print(constant_metrics)
    print('*'*30)

    with open(checkpoint_dir+'metrics.pkl', 'wb') as fp:
        pickle.dump(metrics, fp)

    with open(checkpoint_dir+'constant_metrics.pkl', 'wb') as fp:
        pickle.dump(constant_metrics, fp)



Data and model loaded
Checkpoint dir: ./checkpoint/social-tag/
Training started ...
TRAIN: 	 Epoch: 0 	 Loss: 0.01424446888267994
TRAIN: 	 Epoch: 0 	 Loss: 0.014114801306277514
TRAIN: 	 Epoch: 0 	 Loss: 0.013962577717999617
TRAIN: 	 Epoch: 0 	 Loss: 0.013809907250106335
TRAIN: 	 Epoch: 0 	 Loss: 0.01364209335297346
TRAIN: 	 Epoch: 0 	 Loss: 0.013494737756748995
TRAIN: 	 Epoch: 0 	 Loss: 0.013324068859219551
TRAIN: 	 Epoch: 0 	 Loss: 0.01315830263774842
TRAIN: 	 Epoch: 0 	 Loss: 0.012993430201378133
TRAIN: 	 Epoch: 0 	 Loss: 0.012832795176655054
TRAIN: 	 Epoch: 0 	 Loss: 0.012663833204318176
TRAIN: 	 Epoch: 0 	 Loss: 0.012502921046689153
TRAIN: 	 Epoch: 0 	 Loss: 0.012340806567898164
TRAIN: 	 Epoch: 0 	 Loss: 0.012168123785938536
TRAIN: 	 Epoch: 0 	 Loss: 0.01200572041173776
TRAIN: 	 Epoch: 0 	 Loss: 0.011846053239423782
TRAIN: 	 Epoch: 0 	 Loss: 0.011687614461954902
TRAIN: 	 Epoch: 0 	 Loss: 0.011515091121610668
TRAIN: 	 Epoch: 0 	 Loss: 0.01134265934754359
TRAIN: 	 Epoch: 0 	 Loss: 0.

100%|██████████| 301/301 [00:24<00:00, 12.48it/s]

Testing ....


ADE: 1.381831180121023  FDE: 1.713589116786552
**************************************************
******************************
Epoch: social-tag : 0
train_loss 0.011134542381202798
val_loss 0.028863569004716122
{'min_val_epoch': 0, 'min_val_loss': 0.028863569004716122}
******************************
TRAIN: 	 Epoch: 1 	 Loss: 0.007916895672678947
TRAIN: 	 Epoch: 1 	 Loss: 0.007647848920896649
TRAIN: 	 Epoch: 1 	 Loss: 0.0074487597060700255
TRAIN: 	 Epoch: 1 	 Loss: 0.007221706444397569
TRAIN: 	 Epoch: 1 	 Loss: 0.007074274029582739
TRAIN: 	 Epoch: 1 	 Loss: 0.007005832623690367
TRAIN: 	 Epoch: 1 	 Loss: 0.006859700860721725
TRAIN: 	 Epoch: 1 	 Loss: 0.006744254671502858
TRAIN: 	 Epoch: 1 	 Loss: 0.006622018292546272
TRAIN: 	 Epoch: 1 	 Loss: 0.006487485114485025
TRAIN: 	 Epoch: 1 	 Loss: 0.006371751554648985
TRAIN: 	 Epoch: 1 	 Loss: 0.006249602107952039
TRAIN: 	 Epoch: 1 	 Loss: 0.006155698810918973
TRAIN: 	 Epoch: 1 	 Loss: 0.006052422636587705
TRAIN: 	 Epoch: 1 	 Loss: 0.0059689432

100%|██████████| 301/301 [00:24<00:00, 12.22it/s]

...

Testing ....


ADE: 0.37673412453159694  FDE: 0.5901803069526762
**************************************************
******************************
Epoch: social-tag : 157
train_loss -0.013569002242941258
val_loss -0.012929200742171774
{'min_val_epoch': 157, 'min_val_loss': -0.012929200742171774}
******************************
TRAIN: 	 Epoch: 158 	 Loss: -0.013494055718183517
TRAIN: 	 Epoch: 158 	 Loss: -0.013446212280541658
TRAIN: 	 Epoch: 158 	 Loss: -0.013927657157182693
TRAIN: 	 Epoch: 158 	 Loss: -0.013957092305645347
TRAIN: 	 Epoch: 158 	 Loss: -0.013793038204312325
TRAIN: 	 Epoch: 158 	 Loss: -0.013962949470927319
TRAIN: 	 Epoch: 158 	 Loss: -0.013894646295479365
TRAIN: 	 Epoch: 158 	 Loss: -0.01392193092033267
TRAIN: 	 Epoch: 158 	 Loss: -0.013691699132323265
TRAIN: 	 Epoch: 158 	 Loss: -0.01367619289085269
TRAIN: 	 Epoch: 158 	 Loss: -0.013637560843066736
TRAIN: 	 Epoch: 158 	 Loss: -0.013529695648079118
TRAIN: 	 Epoch: 158 	 Loss: -0.01354643630866821
TRAIN: 	 Epoch: 158 	 Loss: -0.013565312

100%|██████████| 301/301 [00:24<00:00, 12.48it/s]

Testing ....


ADE: 0.3587777940329964  FDE: 0.5329856783310291
**************************************************
******************************
Epoch: social-tag : 202
train_loss -0.013947719402191909
val_loss -0.013223659589094817
{'min_val_epoch': 202, 'min_val_loss': -0.013223659589094817}
******************************
TRAIN: 	 Epoch: 203 	 Loss: -0.01456633023917675
TRAIN: 	 Epoch: 203 	 Loss: -0.014336977154016495
TRAIN: 	 Epoch: 203 	 Loss: -0.014043731614947319
TRAIN: 	 Epoch: 203 	 Loss: -0.014263628050684929
TRAIN: 	 Epoch: 203 	 Loss: -0.014306720532476901
TRAIN: 	 Epoch: 203 	 Loss: -0.014377231088777384
TRAIN: 	 Epoch: 203 	 Loss: -0.014406363080654825
TRAIN: 	 Epoch: 203 	 Loss: -0.014377532759681344
TRAIN: 	 Epoch: 203 	 Loss: -0.014364316335154904
TRAIN: 	 Epoch: 203 	 Loss: -0.014278998970985413
TRAIN: 	 Epoch: 203 	 Loss: -0.014250631782818924
TRAIN: 	 Epoch: 203 	 Loss: -0.014158098570381602
TRAIN: 	 Epoch: 203 	 Loss: -0.01409009607652059
TRAIN: 	 Epoch: 203 	 Loss: -0.014059458

In [ ]:
import os
import math
import sys
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle
import argparse
import glob
import torch.distributions.multivariate_normal as torchdist
import copy

def test(model,loader_test, KSTEPS=20):
    model.eval()
    ade_bigls = []
    fde_bigls = []
    raw_data_dict = {}
    step =0
    for batch in loader_test:
        step+=1
        #Get data
        batch = [tensor for tensor in batch]
        obs_traj, pred_traj_gt, obs_traj_rel, pred_traj_gt_rel, non_linear_ped,\
         loss_mask,V_obs,A_obs,A_dir_obs,V_tr,A_tr,A_dir_tr = batch


        num_of_objs = obs_traj_rel.shape[1]

        #Forward
        #V_obs = batch,seq,node,feat
        #V_obs_tmp = batch,feat,seq,node
        V_obs_tmp =V_obs.permute(0,3,1,2)

        V_pred,_,_ = model(V_obs_tmp,A_obs.squeeze(),A_dir_obs.squeeze())
        # print(V_pred.shape)
        # torch.Size([1, 5, 12, 2])
        # torch.Size([12, 2, 5])
        V_pred = V_pred.permute(0,2,3,1)
        # torch.Size([1, 12, 2, 5])>>seq,node,feat
        # V_pred= torch.rand_like(V_tr).cuda()


        V_tr = V_tr.squeeze()
        A_tr = A_tr.squeeze()
        A_dir_tr = A_dir_tr.squeeze()
        V_pred = V_pred.squeeze()
        num_of_objs = obs_traj_rel.shape[1]
        V_pred,V_tr =  V_pred[:,:num_of_objs,:],V_tr[:,:num_of_objs,:]
        #print(V_pred.shape)

        #For now I have my bi-variate parameters
        #normx =  V_pred[:,:,0:1]
        #normy =  V_pred[:,:,1:2]
        sx = torch.exp(V_pred[:,:,2]) #sx
        sy = torch.exp(V_pred[:,:,3]) #sy
        corr = torch.tanh(V_pred[:,:,4]) #corr

        cov = torch.zeros(V_pred.shape[0],V_pred.shape[1],2,2)
        cov[:,:,0,0]= sx*sx
        cov[:,:,0,1]= corr*sx*sy
        cov[:,:,1,0]= corr*sx*sy
        cov[:,:,1,1]= sy*sy
        mean = V_pred[:,:,0:2]

        mvnormal = torchdist.MultivariateNormal(mean,cov)


        ### Rel to abs
        ##obs_traj.shape = torch.Size([1, 6, 2, 8]) Batch, Ped ID, x|y, Seq Len

        #Now sample 20 samples
        ade_ls = {}
        fde_ls = {}
        V_x = seq_to_nodes(obs_traj.data.cpu().numpy().copy())
        V_x_rel_to_abs = nodes_rel_to_nodes_abs(V_obs.data.cpu().numpy().squeeze().copy(),
                                                 V_x[0,:,:].copy())

        V_y = seq_to_nodes(pred_traj_gt.data.cpu().numpy().copy())
        V_y_rel_to_abs = nodes_rel_to_nodes_abs(V_tr.data.cpu().numpy().squeeze().copy(),
                                                 V_x[-1,:,:].copy())

        raw_data_dict[step] = {}
        raw_data_dict[step]['obs'] = copy.deepcopy(V_x_rel_to_abs)
        raw_data_dict[step]['trgt'] = copy.deepcopy(V_y_rel_to_abs)
        raw_data_dict[step]['pred'] = []

        for n in range(num_of_objs):
            ade_ls[n]=[]
            fde_ls[n]=[]

        for k in range(KSTEPS):

            V_pred = mvnormal.sample()



            #V_pred = seq_to_nodes(pred_traj_gt.data.numpy().copy())
            V_pred_rel_to_abs = nodes_rel_to_nodes_abs(V_pred.data.cpu().numpy().squeeze().copy(),
                                                     V_x[-1,:,:].copy())
            raw_data_dict[step]['pred'].append(copy.deepcopy(V_pred_rel_to_abs))

           # print(V_pred_rel_to_abs.shape) #(12, 3, 2) = seq, ped, location
            for n in range(num_of_objs):
                pred = []
                target = []
                obsrvs = []
                number_of = []
                pred.append(V_pred_rel_to_abs[:,n:n+1,:])
                target.append(V_y_rel_to_abs[:,n:n+1,:])
                obsrvs.append(V_x_rel_to_abs[:,n:n+1,:])
                number_of.append(1)

                ade_ls[n].append(ade(pred,target,number_of))
                fde_ls[n].append(fde(pred,target,number_of))

        for n in range(num_of_objs):
            ade_bigls.append(min(ade_ls[n]))
            fde_bigls.append(min(fde_ls[n]))

    ade_ = sum(ade_bigls)/len(ade_bigls)
    fde_ = sum(fde_bigls)/len(fde_bigls)
    return ade_,fde_,raw_data_dict
#check how our new best saved model performs on test dataset
######################################################################################################
paths = ['/kaggle/working/checkpoint/social-tag']
KSTEPS=20

print("*"*50)
print('Number of samples:',KSTEPS)
print("*"*50)

for feta in range(len(paths)):
    ade_ls = []
    fde_ls = []
    path = paths[feta]
    exps = glob.glob(path)
    print('Model being tested are:',exps)

    for exp_path in exps:
        print("*"*50)
        print("Evaluating model:",exp_path)

        model_path = exp_path+'/val_best.pth'
        args_path = exp_path+'/args.pkl'
        with open(args_path,'rb') as f:
            args = pickle.load(f)

        #Data prep
        obs_seq_len = args.obs_seq_len
        pred_seq_len = args.pred_seq_len
        data_set = '/kaggle/input/dataset/datasets/'+args.dataset+'/'

        dset_test = TrajectoryDataset(
                data_set+'test/',
                obs_len=obs_seq_len,
                pred_len=pred_seq_len,
                skip=1,norm_lap_matr=True)

        loader_test = DataLoader(
                dset_test,
                batch_size=1,#This is irrelative to the args batch size parameter
                shuffle =False,
                num_workers=1)



        #Defining the model
        modelka = social_stgcnn(n_stgcnn =args.n_stgcnn,n_txpcnn=args.n_txpcnn,
        output_feat=args.output_size,seq_len=args.obs_seq_len,
        kernel_size=args.kernel_size,pred_seq_len=args.pred_seq_len)
        modelka.load_state_dict(torch.load(model_path))


        ade_ =999999
        fde_ =999999
        print("Testing ....")
        ad,fd,raw_data_dic_= test(modelka,loader_test)
        ade_= min(ade_,ad)
        fde_ =min(fde_,fd)
        ade_ls.append(ade_)
        fde_ls.append(fde_)
        print("ADE:",ade_," FDE:",fde_)




    print("*"*50)
############################################################################################################################

**************************************************
Number of samples: 20
**************************************************
Model being tested are: ['/kaggle/working/checkpoint/social-tag']
**************************************************
Evaluating model: /kaggle/working/checkpoint/social-tag
Processing Data .....


100%|██████████| 301/301 [00:23<00:00, 12.55it/s]

Testing ....


ADE: 0.3588602304791608  FDE: 0.53796490825129
**************************************************
